## XGBSurv Breslow Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/gbdt_pipeline')
from gbdt_pipeline import get_gbdt_pipeline, train_gbdt, train_gbdt_complete

current path /Users/JUSC/Documents/xgbsurv/experiments/breslow


## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 50
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_gbdt_'


# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [4]:
data_set_fns = [ load_metabric, load_flchain, load_rgbsg, load_support]  # 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'breslow'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.635204974571211
Integrated Brier Score: 0.16408580398423547
{'model': 'breslow', 'dataset': 'METABRIC', 'cindex_train': [0.6309594855273617], 'cindex_test': [0.635204974571211], 'ibs_train': [0.16584586462214745], 'ibs_test': [0.16408580398423547]}
1
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5334767140358988
Integrated Brier Score: 0.17692111889468995
{'model': 'breslow', 'dataset': 'METABRIC', 'cindex_train': [0.6346963749740127], 'cindex_test': [0.5334767140358988], 'ibs_train': [0.16208003570866555], 'ibs_test': [0.17692111889468995]}
2
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6434277558625885
Integrated Brier Score: 0.16812005551111983
{'model': 'breslow', 'dat

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,breslow,METABRIC,0.630959,0.635205,0.165846,0.164086
1,breslow,METABRIC,0.634696,0.533477,0.162080,0.176921
2,breslow,METABRIC,0.672501,0.643428,0.153729,0.168120
3,breslow,METABRIC,0.646746,0.637451,0.154708,0.163868
4,breslow,METABRIC,0.672767,0.645871,0.155115,0.170077
5,breslow,FLCHAIN,0.794749,0.803548,0.098277,0.099230
6,breslow,FLCHAIN,0.800745,0.774973,0.097506,0.102674
7,breslow,FLCHAIN,0.743937,0.740108,0.105380,0.106901
8,breslow,FLCHAIN,0.794667,0.800126,0.099899,0.096225
9,breslow,FLCHAIN,0.800472,0.787357,0.097030,0.097745


## TCGA Train, Test, Evaluation

In [9]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'breslow'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5193370165745856
Integrated Brier Score: 0.2453954499687396
{'model': 'breslow', 'dataset': 'BLCA', 'cindex_train': [0.8081787676536746], 'cindex_test': [0.5193370165745856], 'ibs_train': [0.1821776350957438], 'ibs_test': [0.2453954499687396]}
1
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.4959038776624795
Integrated Brier Score: 0.22508273965206654
{'model': 'breslow', 'dataset': 'BLCA', 'cindex_train': [0.5049897978111667], 'cindex_test': [0.4959038776624795], 'ibs_train': [0.2009594951385878], 'ibs_test': [0.22508273965206654]}
2
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5734767025089605
Integrated Brier Score: 0.22860487040988342
{'model': 'breslow', 'dataset': 'BLCA',

In [15]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'tcga_results.csv',index=False)

In [14]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,breslow,BLCA,0.808179,0.519337,0.182178,0.245395
1,breslow,BLCA,0.504990,0.495904,0.200959,0.225083
2,breslow,BLCA,0.922495,0.573477,0.168613,0.228605
3,breslow,BLCA,0.776270,0.550974,0.156860,0.321074
4,breslow,BLCA,0.935602,0.592760,0.129129,0.210455
5,breslow,BRCA,0.801810,0.427083,0.175365,0.211416
6,breslow,BRCA,0.491094,0.225846,0.187460,0.190356
7,breslow,BRCA,0.678141,0.523507,0.182555,0.193338
8,breslow,BRCA,0.691936,0.499826,0.172922,0.178389
9,breslow,BRCA,0.902592,0.625244,0.173351,0.200452
